In [1]:
import numpy as np
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
df = pd.read_csv("latimes-county-totals.csv") 
print (df.head(5))
df.describe()

         date   county  fips  confirmed_cases  deaths  new_confirmed_cases  \
0  2020-01-26  Alameda     1                0       0                  NaN   
1  2020-01-31  Alameda     1                0       0                  0.0   
2  2020-02-02  Alameda     1                0       0                  0.0   
3  2020-02-20  Alameda     1                0       0                  0.0   
4  2020-02-21  Alameda     1                0       0                  0.0   

   new_deaths  
0         NaN  
1         0.0  
2         0.0  
3         0.0  
4         0.0  


,fips,confirmed_cases,deaths,new_confirmed_cases,new_deaths
count,2485.000000,2485.000000,2485.000000,2432.000000,2432.000000
mean,59.756539,229.698189,6.879276,15.485197,0.591283
std,32.786933,974.019813,37.795395,70.478349,3.502466
min,1.000000,0.000000,0.000000,-2.000000,0.000000
25%,31.000000,2.000000,0.000000,0.000000,0.000000
50%,65.000000,14.000000,0.000000,1.000000,0.000000
75%,85.000000,101.000000,2.000000,7.000000,0.000000
max,115.000000,16449.000000,732.000000,1474.000000,76.000000


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2485 entries, 0 to 2484
Data columns (total 7 columns):
date                   2485 non-null object
county                 2485 non-null object
fips                   2485 non-null int64
confirmed_cases        2485 non-null int64
deaths                 2485 non-null int64
new_confirmed_cases    2432 non-null float64
new_deaths             2432 non-null float64
dtypes: float64(2), int64(3), object(2)
memory usage: 136.0+ KB


In [4]:
def SEIRD_model(N, E, I, r, r2, re, r2e, p, d, shelter_in_place, b, b2, y, a, days):
    '''
    N - Total population，N = I + S + R + E 
    S - Number of people that are Susceptibles
    R - Number of people that are Recovered
    E - Number of people that are Exposed
    I - Number of people that are Infected
    r - The number of people each infected contacts each day
    re - New r after shelter in place
    r2 - The number of people each asymptomatic contacts each day
    r2e - New r2 after shelter in place takes effect
    p - Protection rate of the masks. After shelter in place, b=b*p, b2 = b2*p
    d - The proportion of infected who die each day to the total number of infected people
    shelter_in_place - Days since shelter in place
    b - Probability of transmission from infected to susceptible person
    b2 Probability of transmission from asymptomatic to susceptible
    y - The proportion of infected who are recovered each day to the total number of infected people
    a - Probability of susceptible transforming to infected
    days - Duration in days
    '''
    S = N - I - E
    I_list = [I]
    S_list = [S]
    E_list = [E]
    R_list = [0]
    D_list = [0]
    C_list = [0] # Confirmed list 

    for i in range(1, days):
        if (i + 1) == shelter_in_place:
            r, r2, b, b2 = re, r2e, b * p, b2 * p
        E_perday = r * b * I_list[i - 1] * S_list[i - 1] / N
        E_perday_for_E = r2 * b2 * E_list[i - 1] * S_list[i - 1] / N
        I_perday = a * E_list[i - 1]
        R_perday = y * I_list[i - 1]
        D_perday = d * I_list[i - 1]
        C_list.append(C_list[i - 1] + I_perday)
        I_list.append(I_list[i - 1] + I_perday - R_perday - D_perday)
        S_list.append(S_list[i - 1] - E_perday - E_perday_for_E)
        R_list.append(R_list[i - 1] + R_perday)
        D_list.append(D_list[i - 1] + D_perday)
        E_list.append(E_list[i - 1] + E_perday + E_perday_for_E - I_perday)
    return(D_list)

In [5]:
result = SEIRD_model(10000, 0, 1, 20, 20, 5, 5, 0.8, 0.01, 10, 0.03, 0.03, 0.1, 0.1, 140)

In [6]:
print(result)

[0, 0.01, 0.0189, 0.02742094, 0.036438343571123785, 0.047125397582657116, 0.06116944080034882, 0.08110669306534783, 0.11084560450732503, 0.15648930081716433, 0.2276299999789867, 0.322605829803142, 0.44026298379822176, 0.5798857148099723, 0.7411397238712237, 0.9240267243352901, 1.1288483478263835, 1.356177878221726, 1.6068385641753782, 1.8818874790206153, 2.1826040770224813, 2.510482743309253, 2.8672287566737973, 3.2547571841457295, 3.675194307353303, 4.130881246129468, 4.624379496936452, 5.158478144402265, 5.736202535118192, 6.360824225061785, 7.035872026546907, 7.765143988206818, 8.552720142757252, 9.402975852591485, 10.320595572946804, 11.310586836691332, 12.378294243903147, 13.52941321352446, 14.770003223661947, 16.10650023181056, 17.545727926731587, 19.094907420375822, 20.761664941744513, 22.554037045782152, 24.480472800426725, 26.549832365257803, 28.771381327602352, 31.154780118713738, 33.710067796399905, 36.447639454360456, 39.37821650605318, 42.51280909609111, 45.86266991921512,